# Wav2Vec2 모델을 사용한 한국어 음성 인식
이 노트북에서는 Hugging Face의 Wav2Vec2 모델을 사용하여 한국어 음성을 텍스트로 변환합니다.

In [1]:
# 필요한 라이브러리 임포트
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torchaudio
from jamo import h2j, j2hcj, is_jamo

## 모델 및 프로세서 로드
사전 훈련된 Wav2Vec2 모델과 프로세서를 Hugging Face Hub에서 로드합니다.

In [2]:
# 모델과 프로세서 로드
model_name = "Kkonjeong/wav2vec2-base-korean"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# 모델을 평가 모드로 설정
model.eval()
model.to("cuda")

## 자모 결합 함수
자모(Jamo)를 결합하여 완전한 한글 텍스트로 변환하는 함수를 정의합니다.

In [3]:
# 자모 결합 함수
def combine_jamo(jamo_text):
    hangul_text = ""
    buffer = []

    # 자모 유효성 검사 함수
    def is_valid_jamo(jamo):
        return 0x1100 <= ord(jamo) <= 0x11FF

    def flush_buffer(buffer):
        if len(buffer) == 3:
            initial = ord(buffer[0]) - 0x1100
            medial = ord(buffer[1]) - 0x1161
            final = ord(buffer[2]) - 0x11A7
            if (0 <= initial < 19) and (0 <= medial < 21) and (0 <= final < 28):
                return chr(0xAC00 + initial * 588 + medial * 28 + final)
        elif len(buffer) == 2:
            initial = ord(buffer[0]) - 0x1100
            medial = ord(buffer[1]) - 0x1161
            if (0 <= initial < 19) and (0 <= medial < 21):
                return chr(0xAC00 + initial * 588 + medial * 28)
        elif len(buffer) == 1:
            return buffer[0]
        return ''.join(buffer)

    for jamo in jamo_text:
        if is_jamo(jamo) and is_valid_jamo(jamo):
            buffer.append(jamo)
            if len(buffer) == 3:
                hangul_text += flush_buffer(buffer)
                buffer = []
        else:
            hangul_text += flush_buffer(buffer)
            hangul_text += jamo
            buffer = []

    if buffer:
        hangul_text += flush_buffer(buffer)

    return hangul_text

## 음성 파일로부터 텍스트 추론 함수
음성 파일을 입력으로 받아서 모델을 통해 텍스트를 추론하는 함수를 정의합니다.

In [4]:
# 음성 파일을 입력으로 받아 추론을 수행하는 함수
def predict_from_audio(audio_path):
    # 음성 파일 로드
    speech_array, sampling_rate = torchaudio.load(audio_path)

    # 샘플링 레이트가 16000Hz가 아닌 경우 리샘플링
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        speech_array = resampler(speech_array)

    # 2D tensor로 변환
    speech_array = speech_array.squeeze().numpy()

    # 음성 파일 전처리
    input_values = processor(speech_array, sampling_rate=16000, return_tensors="pt").input_values
    input_values = input_values.to("cuda")

    # 모델 추론
    with torch.no_grad():
        logits = model(input_values).logits

    # 자모 텍스트 디코딩
    pred_ids = torch.argmax(logits, dim=-1)
    jamo_text = processor.batch_decode(pred_ids)[0]

    # 자모 텍스트 결합하여 최종 문장 생성
    final_text = combine_jamo(jamo_text)

    return final_text

## 예시: 음성 파일로부터 텍스트 추론
지정된 음성 파일을 사용하여 텍스트를 추론합니다.

In [5]:
# 예시: 음성 파일로부터 텍스트 추론
audio_path = "jiwon_.wav"
predicted_text = predict_from_audio(audio_path)
print("Predicted Text: ", predicted_text)

Predicted Text:  ㅁㅣㄱㅏ ㅇㅔㅆㅇㅓㅈㅜㄴ ㄱㅓㅅ ㅊㅏㄹㅏ ㅇㅏㄹㄱㅗ ㅍㅜㄴㅓ
